In [1]:
!pip install prophet

StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 3, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 137.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 38.4 MB/s eta 0:00:00


In [2]:
import pandas as pd 
from prophet import Prophet
from datetime import datetime

StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 4, Finished, Available)

In [3]:
# read in processed water level data 
df = pd.read_csv("/lakehouse/default/" + "Files/dim_fact_copies/vw_ml_training_flood_short.csv")
df.head()


StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 5, Finished, Available)

,flood_area_key,flood_area_label,flood_area_latitude,flood_area_longitude,date,rainfall,water_level_typical_high,water_level_high,water_level_low,water_level_daily_fluctuation,flood
0,211,"River Douglas at Parbold, along the right bank...",53.59066,-2.77339,2023-11-07 00:00:00.0000000,0.0,2.925,0.000,0.000,0.000,0
1,220,"River Calder at Fulledge, area surrounding Mit...",53.78635,-2.22992,2023-06-24 00:00:00.0000000,0.0,1.200,0.000,0.000,0.000,0
2,157,"Lancashire coastline at Clifton Marsh, adjacen...",53.74476,-2.85869,2023-10-05 00:00:00.0000000,0.0,1.712,0.000,0.000,0.000,0
3,125,Coast at North Morecambe Bay,54.20247,-2.91313,2024-01-06 00:00:00.0000000,19.2,1.200,0.723,0.641,0.082,0
4,16,"River Eden at Warwick Bridge, Warwick Park and...",54.89469,-2.83336,2023-03-30 00:00:00.0000000,0.0,3.360,0.000,0.000,0.000,0


In [4]:
df['date'] = pd.to_datetime(df['date'])

# filter to latest data which is more complete
df = df[df['date'] > datetime.strptime('2024-01-30', '%Y-%m-%d')]

# rename columns for Prophet 
df.rename(columns={'date':'ds', 'water_level_high':'y'}, inplace=True)

StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 6, Finished, Available)

In [6]:
# remove stations that are only ever 0 for computation speed reasons

key_zero = df[df['y']==0]['flood_area_key'].unique()
key_nonzero = df[df['y']!=0]['flood_area_key'].unique()

key_only_zero = set(key_zero) - set(key_nonzero)

df_final = df[~df['flood_area_key'].isin(key_only_zero)]

StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 8, Finished, Available)

In [7]:
# read in data to filter by area
flood_area = pd.read_csv("/lakehouse/default/" + "Files/dim_fact_copies/dim_flood_area.csv")

# calculating risk just for Yorkshire for computation speed reasons
filt = flood_area['county'].str.contains('Yorkshire')

subsection = flood_area[filt]['flood_area_key'].unique()

StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 9, Finished, Available)

In [8]:
# loop through measurement stations and make a 5 day forecast of water levels

preds = []

for group, group_df in df_final[df_final['flood_area_key'].isin(subsection)].groupby('flood_area_key'):
    m = Prophet()
    m.fit(group_df[['ds', 'y']])
    future = m.make_future_dataframe(periods=5)
    forecast = m.predict(future)
    forecast['flood_area_key'] = group
    preds.append(forecast)

StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 10, Finished, Available)

11:41:34 - cmdstanpy - INFO - Chain [1] start processing
11:41:34 - cmdstanpy - INFO - Chain [1] done processing
11:41:34 - cmdstanpy - INFO - Chain [1] start processing
11:41:34 - cmdstanpy - INFO - Chain [1] done processing
11:41:34 - cmdstanpy - INFO - Chain [1] start processing
11:41:34 - cmdstanpy - INFO - Chain [1] done processing
11:41:34 - cmdstanpy - INFO - Chain [1] start processing
11:41:35 - cmdstanpy - INFO - Chain [1] done processing
11:41:35 - cmdstanpy - INFO - Chain [1] start processing
11:41:35 - cmdstanpy - INFO - Chain [1] done processing
11:41:35 - cmdstanpy - INFO - Chain [1] start processing
11:41:35 - cmdstanpy - INFO - Chain [1] done processing
11:41:35 - cmdstanpy - INFO - Chain [1] start processing
11:41:35 - cmdstanpy - INFO - Chain [1] done processing
11:41:35 - cmdstanpy - INFO - Chain [1] start processing
11:41:35 - cmdstanpy - INFO - Chain [1] done processing
11:41:35 - cmdstanpy - INFO - Chain [1] start processing
11:41:35 - cmdstanpy - INFO - Chain [1]

Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
n_changepoints greater than number of observations. Using 23.
input tempfile: /tmp/tmpeg0lnp6s/lfqrvg_6.json
input tempfile: /tmp/tmpeg0lnp6s/dxs41yt6.json
idx 0
running CmdStan, num_threads: None
CmdStan args: ['/home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83420', 'data', 'file=/tmp/tmpeg0lnp6s/lfqrvg_6.json', 'init=/tmp/tmpeg0lnp6s/dxs41yt6.json', 'output', 'file=/tmp/tmpeg0lnp6s/prophet_modelve8hevkw/prophet_model-20240304114134.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
Chain [1] start processing
Chain [1] done processing
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
n_changepo

In [9]:
subsection_output = pd.concat(preds)

subsection_output.to_csv("/lakehouse/default/" + "Files/water_level_predictions.csv")

StatementMeta(, dc0a37f5-298e-4ef2-ac38-9a1b6ce86a4c, 11, Finished, Available)